In [ ]:
import numpy as np

In [ ]:
DATA_URL = '/content/drive/My Drive/food_data/Test_X_array.npy'
data = np.load(DATA_URL)
data.shape

(32573, 208)

In [ ]:
DATA_URL = '/content/drive/My Drive/food_data/Train_X_Array_ver2.npy'
train_data = np.load(DATA_URL)
train_data.shape

(456548, 208)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_URL = '/content/drive/My Drive/food_data/Yarray.npy'
y_data = np.load(DATA_URL)
y_data.shape

(456548,)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn import metrics

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y_data, test_size=0.2, random_state=42)

# **LINEAR REGRESSION**

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
predict_train = lin_reg.predict(X_train)
rmse_train = mean_squared_error(y_train,predict_train)**(0.5)
print('RMSE on train dataset : ', rmse_train)

RMSE on train dataset :  345.56758332840855


In [ ]:
predict_test = lin_reg.predict(X_test)
rmse_test = mean_squared_error(y_test,predict_test)**(0.5)
print('RMSE on test dataset : ', rmse_test)

RMSE on test dataset :  2129589518217.4717


In [ ]:
print("MSE on test dataset : ", mean_squared_error(y_test, predict_test))

MSE on test dataset :  4.535151516101723e+24


# **RANDOM FOREST**

In [ ]:
ran_reg = RandomForestRegressor(random_state=1)
ran_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [ ]:
pred_rF = ran_reg.predict(X_test)
print("Mean Squared Log Error is ", mean_squared_log_error(y_test, pred_rF))

Mean Squared Log Error is  0.4301229983475836


In [ ]:
print("RMSLE:", mean_squared_log_error(y_test, pred_rF)**(0.5))

RMSLE: 0.6558376310853042


In [ ]:
print("RMSE on test dataset : ", mean_squared_error(y_test, pred_rF)**(0.5))

RMSE on test dataset :  266.4772899784216


In [ ]:
metrics.r2_score(y_test, pred_rF)

0.5329492386101725

# **XGBOOST**

In [ ]:
from xgboost import XGBRegressor 
XG = XGBRegressor() 
XG.fit(X_train, y_train) 
y_pred = XG.predict(X_test) 
y_pred[y_pred<0] = 0 
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, y_pred)))

[05:38:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSLE: 106.27689509854181


In [ ]:
pred_xg = XG.predict(X_test)
print("MAE : ", metrics.mean_absolute_error(y_test, pred_xg))

MAE :  165.73204455126574


In [ ]:
print("RMSE on test dataset : ", mean_squared_error(y_test, pred_xg)**(0.5))

RMSE on test dataset :  325.4661677086487


In [ ]:
from sklearn.metrics import mean_squared_error as MSE
rmse = np.sqrt(MSE(y_test, pred_xg))
print("RMSE : % f" %(rmse))

RMSE :  325.466168


In [ ]:
metrics.r2_score(y_test, pred_xg)

0.3032846502655696

In [ ]:
n_estimators = [100, 500, 900, 1000, 1100, 1500]
max_depth = [2, 3, 5, 7, 10, 15]
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]
learning_rate=[0.05,0.1,0.13,0.15,0.17,0.20]
min_child_weight=[1,2,3,4,5,9,10]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [ ]:
 from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_cv = RandomizedSearchCV(estimator=XG,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [ ]:
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
